<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clustering chapter. Example of unsupervised learning.

# Only do this once per VM, otherwise you'll get multiple clones and nested directories
!git clone https://github.com/jamestheengineer/data-science-from-scratch-Python.git
%cd data-science-from-scratch-Python/
!pip install import-ipynb
import import_ipynb

In [ ]:
from Chapter_04 import Vector

def num_differences(v1: Vector, v2: Vector) -> int:
  assert len(v1) == len(v2)
  return len([x1 for x1, x2 in zip(v1,v2) if x1 != x2])

assert num_differences([1,2,3],[2,1,3]) == 2
assert num_differences([1,2],[1,2]) == 0

In [ ]:
from typing import List
from Chapter_04 import vector_mean

def cluster_means(k: int,
                  inputs: List[Vector],
                  assignments: List[int]) -> List[Vector]:
  # cluster[i] contains the inputs whose assignment is i
  clusters = [[] for i in range(k)]
  for input, assignment in zip(inputs, assignents):
    clusters[assignment].append(input)

  # if a cluster is empty, just use a random point
  return [vector_mean(cluster) if cluster else random.choice(inputs)
          for cluster in clusters]

In [ ]:
import itertools
import random
import tqdm
from Chapter_04 import squared_distance

class KMeans:
  def __init__(self, k: int) -> None:
    self.k = k
    self.means = None

  def classify(self, input: Vector) -> int:
    """return the index of the cluster closes to the input"""
    return min(range(self.k),
               key=lambda i: squared_distance(input, self.means[i]))
    
  def train(self, inputs: List)